In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr
import yaml

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import acr.utils as acu
import plotly.express as px
plt.style.use('acr_plots.mplstyle')

In [37]:
#quick editing/exploration
dp = f'/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ACR_12/subject_info.yml'
with open(dp) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
data = {}

In [ ]:
params = {}
params['subject'] = 'ACR_'
params['exps'] = []
params['stores'] = []
params['channels'] = {}

In [56]:
float(t['control1'][2])

99445.862213

In [ ]:
with open(dp, "w") as f:
    yaml.dump(data, f)

In [57]:
def get_rec_times(params):
    times = {}
    sub = params['subject']
    for exp in params['exps']:
        p = acr.io.acr_path(sub, exp)
        d = tdt.read_block(p, t1=0, t2=1, evtype=["scalars"])
        i = d.info
        start = np.datetime64(i.start_date)
        end = np.datetime64(i.stop_date)
        d = (end - start) / np.timedelta64(1, "s")
        times[exp] = (str(start.astype(str)), str(end.astype(str)), float(d))
    return times

def subject_info_gen(params):
    """Params is a dictionary with the following keys:
    subject: str
    exps: list of experiment keys
    stores: list of relevant data stores, to be used in preprocessing of raw data
    channels: a dictionary whose keys are values from stores, and values are the channels to use for each store
    """
    subject = params['subject']
    exps = params['exps']
    stores = params['stores']
    channels = params['channels']
    path = f'/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/{subject}/subject_info.yml'
    with open(path) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    data = {}
    data['subject'] = subject
    times = get_rec_times(params)
    data['rec_times'] = times
    data['channels'] = channels
    data['paths'] = acr.io.get_acr_paths(subject, exps)
    data['stores'] = stores
    with open(path, "w") as f:
        yaml.dump(data, f)

In [5]:
path_to_pm = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/'
subject = 'ACR_12'
stores = ['EEGr', 'EMGr', 'NNXr']
channels = {'EEGr': [1, 2], 'EMGr': [1, 2], 'NNXr': list(range(1, 17))}
exps = ['control1', 'laser1']

In [14]:
params = {}
params['subject'] = 'ACR_12'
params['exps'] = ["control1-bl", "laser1-bl", "control1", "laser1"]
params['stores'] = [{'EEGr': [1, 2], 'EMGr': [1, 2], 'NNXr': list(range(1, 17))}]
params['channels'] = {}
subject_info_gen()

In [15]:
t = get_rec_times(exps, subject)

read from t=0s to t=1617.32s
read from t=0s to t=1651.55s


In [38]:
data['recording_times'] = {}
for key in exps:
    data['recording_times'][key] = {}
    data['recording_times'][key]['start'] = str(t[key][0])
    data['recording_times'][key]['end'] = str(t[key][1])

In [41]:
data['subject'] = subject

In [44]:
data['paths'] = acr.io.get_acr_paths(subject, exps)

In [46]:
data['channels'] = {}
for store in stores:
    data['channels'][store] = channels[store]

In [43]:
acr.io.get_acr_paths(subject, exps)

{'control1': '/Volumes/opto_loc/Data/ACR_12/ACR_12-control1',
 'laser1': '/Volumes/opto_loc/Data/ACR_12/ACR_12-laser1'}

In [32]:
type(t['control1'][0])

numpy.str_

In [36]:
type(str(t['control1'][0]))

str